<a href="https://colab.research.google.com/github/HuyenNguyenHelen/Huyen_INFO5731_Spring2020/blob/master/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [2019 Dell labtop](https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/dp/B07N49F51N/ref=sr_1_11?crid=1IJ7UWF2F4GHH&keywords=dell%2Bxps%2B15&qid=1580173569&sprefix=dell%2Caps%2C181&sr=8-11&th=1) on amazon.

(2) Collect the top 100 User Reviews of the film [Joker](https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv) from IMDB.

(3) Collect the abstracts of the top 100 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(4) Collect the top 100 tweets by using hashtag ["#wuhancoronovirus"](https://twitter.com/hashtag/wuhancoronovirus) from Twitter. 


In [0]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

In [65]:
# Question1

#Yesss I fixed that problem

host_url = ["https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv"]
load_more_urls = ["https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7crmqm2dczaa7wxhfmzzrdtmsbzhzfmxvlnomwklyczuf43o6ss6oe2fjmzldj4k5tm5fjyr3a2pn6woclsokk47cma", "https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cjmqqzde3yk7swhhmjqrxt4ohjjtzpwzouokkd2gbzgpnt6ucc2peyv5mbjb4dwdpdqzcxjfmg3k65z4tqdaz44w", "https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cbkqmzdizyc76wxvnzrrxs46bzhzfmxvlnomwklyczuf43o6ss6oe4vtpridv4k5vz4e43vvsqoskmtj4psbfq2qni", "https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cbpryzd6yab62tx7obtrtt4objhzfmxvlnomwklyczuf43o6ss6oe4f5nzldn4k4k7d72vy22axhza7lis2c2y7jzi"]
all_urls = host_url+load_more_urls
user_names, user_links, content, date, title, review_helpfulness = [],[], [], [], [], []
review_elements = []
for url in all_urls:
  request = urllib.request.urlopen (url)
  webpage  = request.read()
  soup = BeautifulSoup(webpage)

  reviews = soup.find_all("div", {"class":"lister-item-content" })
  for review in reviews:
    try:
        review_title = review.find("a", class_="title").text.strip()
    except:
        review_title = ""
    try:
        review_id = review.find("a")["href"].split("/")[2]
    except:
        review_id = ""
    try:
        review_date = review.find("span", class_="review-date").get_text()
    except:
        review_date = ""
    try:
        review_nickname = review.find ("span", class_="display-name-link").get_text()
    except:
        review_nickname = ""
    try:
        review_content = review.find ("div", class_="text show-more__control").get_text()
    except:
        review_content =""
    #print(review_content)
    try:
        review_rating = review. find ("span", class_="rating-other-user-rating").getText().strip()
        #print(review_rating)
    except:
        review_rating = ""
    try:
        review_helpfulness = review.find("div", {"class": "actions text-muted"}).text.split("\n")[1]
        #print(review_helpfulness)
    except:
        review_helpfulness = ""

    review_elements.append([review_title, review_id, review_date, review_nickname, review_content, review_rating, review_helpfulness])
#print(len(review_elements))

df = pd.DataFrame(review_elements, columns=["review_title", "review_id", "review_date", "review_nickname", "review_content", "review_rating", "review_helpfulness"])
df

with open("E:\Helen\INFO5731_Assignment2\Joke_reviews.csv", "w", newline="", encoding='utf-8') as file:
  df.to_csv(file)
with open("E:\Helen\INFO5731_Assignment2\Joke_reviews.csv", "r", encoding='utf-8') as file:
  pd.read_csv(file)



115


In [14]:
#try_codes
load_more_data = soup.find_all("div", attrs={"class": "load-more-data"}) # find this button at the end of each page
#print(load_more_data)
for item in load_more_data:
  path_parse1 = "https://www.imdb.com"
  path_parse2 = "/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey="
  path_parse3 = item.get("data-key")
  concatenated_path = path_parse1+path_parse2+path_parse3
  print(concatenated_path)
#load_more_links = ["https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7crmqm2dczaa7wxhfmzzrdtmsbzhzfmxvlnomwklyczuf43o6ss6oe2fjmzldj4k5tm5fjyr3a2pn6woclsokk47cma", "https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cjmqqzde3yk7swhhmjqrxt4ohjjtzpwzouokkd2gbzgpnt6ucc2peyv5mbjb4dwdpdqzcxjfmg3k65z4tqdaz44w", "https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cbkqmzdizyc76wxvnzrrxs46bzhzfmxvlnomwklyczuf43o6ss6oe4vtpridv4k5vz4e43vvsqoskmtj4psbfq2qni", "https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cbpryzd6yab62tx7obtrtt4objhzfmxvlnomwklyczuf43o6ss6oe4f5nzldn4k4k7d72vy22axhza7lis2c2y7jzi"]
#for link in load_more_links:
 # print(link) 

https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7crmqu2ta3qb7oux7nrvqpr42brhzfmxvlnomwklyczuf43o6ss6oe2fjmzldj4k4rmwb65sqjd2c5p4ic5kj7njmla


In [44]:
# In this line, I am attempting to generate links with data-key in the class load-more-data. However, because, the server prohibits to get key in the second next page, so my code raise error.
host_url = "https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv"
def link_load_more (host_url):
  request = urllib.request.urlopen (host_url)
  webpage  = request.read()
  soup = BeautifulSoup(webpage)
  load_more_data = soup.find_all("div", attrs={"class": "load-more-data"}) # find this button at the end of each page
  flag = True
  if len (load_more_data):
  #print(load_more_data)
    for item in load_more_data:
      path_parse1 = "https://www.imdb.com/"
      path_parse2 = "title/tt7286456/reviews/_ajax"
      path_parse3 = "?ref_=undefined&paginationKey="
      try:
        path_parse4 = item.get("data-key")
        concatenated_path = path_parse1+path_parse2+path_parse3+path_parse4
      except KeyError:
        flag = False
  return concatenated_path

# generate link and parse all of four pages

url_nextpage1 = link_load_more(host_url)
url_nextpage2 = link_load_more(url_nextpage1)
url_nextpage3 = link_load_more(url_nextpage2)
url_nextpage4 = link_load_more(url_nextpage3)

print(url_nextpage1)
print(url_nextpage2)
print(url_nextpage3)
print(url_nextpage4)



UnboundLocalError: ignored

# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [88]:
# Write your code here
#clean the text data
from __future__ import division
import nltk # re, pprint
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string 

with open("E:\Helen\INFO5731_Assignment2\Joke_reviews.csv", "r", encoding='utf-8') as file:
  data = pd.read_csv(file)

raw_title = [str(line) for line in data.review_title]
raw_content = [str(line) for line in data.review_content]
# Tokenize raw text in raw_title and raw_content
tokens_title= [nltk.word_tokenize(title) for title in raw_title]
tokens_content = [nltk.word_tokenize(item) for item in raw_content]

# PROCCESSES OF CLEANING TEXT  
def cleaning_text (text):
    step1 = remove_noise(text)
    step2 = remove_num (step1)
    step3 = remove_stop (step2)
    step4 = lowercase (step3)
    step5 = stemming (step4)
    step6 = lemma (step5)
    return step6

# (1) Remove noise, such as special characters and punctuations.      
def remove_noise (list):
    punct=[char for char in string.punctuation]
    noise_removed = [word for word in list if word not in punct]
    return noise_removed

#(2) Remove numbers.
def remove_num (list):
    num_removed  = [word for word in list if not word.isnumeric()]
    return num_removed

 # (3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).
def remove_stop (list):
    stopword_removed = [word for word in list if word not in stopwords.words('english')]
    return stopword_removed

#(4) Lowercase all texts
def lowercase (list):
    lowercase = [word.lower() for word in list]
    return lowercase
# (5) Stemming
def stemming (list):
    ps = PorterStemmer()
    word_stemmed = [ps.stem(word) for word in list]
    return word_stemmed
# (6) Lemmatization.
def lemma (list):
    wnl =  WordNetLemmatizer ()
    word_lemmatized = [wnl.lemmatize(word) for word in list]
    return word_lemmatized

# Cleaning title reviews and content review
cleaned_title = [cleaning_text (sublist) for sublist in tokens_title]
cleaned_content = [cleaning_text (sublist) for sublist in tokens_content]
# Save cleaned text into new columns in dataframe
df["cleaned_title"] = cleaned_title
df["cleaned_content"] = cleaned_content
df






[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,review_title,review_id,review_date,review_nickname,review_content,review_rating,review_helpfulness,cleaned_title,cleaned_content
0,As a viewer that actually went to TIFF and wit...,rw5112402,10 September 2019,JF500,I was a person that saw all the hype and claim...,10/10,"6,837 out of 7,858 found t...","[a, viewer, actual, went, tiff, wit, film, n't...","[i, person, saw, hype, claim, masterpiec, over..."
1,Outstanding movie with a haunting performance ...,rw5159304,3 October 2019,MihaVrhunc,"Every once in a while a movie comes, that trul...",10/10,"3,375 out of 4,039 found t...","[outstand, movi, haunt, perform, best, charact...","[everi, movi, come, truli, make, impact, joaqu..."
2,Only certain people can relate,rw5168360,7 October 2019,lesterarnoldpinto,This is a movie that only those who have felt ...,10/10,"2,673 out of 3,299 found t...","[onli, certain, peopl, relat]","[thi, movi, felt, alon, isol, truli, relat, yo..."
3,Must have put a SMILE of Satisfaction on Heath...,rw5092831,1 September 2019,Chandler_Bing_,"Truly a masterpiece, The Best film of 2019, on...",10/10,"3,166 out of 3,934 found t...","[must, put, smile, satisfact, heath, ledger, '...","[truli, masterpiec, the, best, film, one, best..."
4,The Hype is real,rw5160204,4 October 2019,kdagoulis26,Most of the time movies are anticipated like t...,10/10,"2,352 out of 2,918 found t...","[the, hype, real]","[most, time, movi, anticip, like, end, fall, s..."
...,...,...,...,...,...,...,...,...,...
110,So disappointed by this film,rw5187643,14 October 2019,huntj-59716,Was really looking forward to seeing this film...,2/10,71 out of 139 found this h...,"[so, disappoint, film]","[wa, realli, look, forward, see, film, howev, ..."
111,Well filmed but empty,rw5179068,11 October 2019,Red_Identity,What can I say about it that hasn't already be...,,41 out of 76 found this he...,"[well, film, empti]","[what, i, say, n't, alreadi, said, it, 's, ter..."
112,"Spare me the ""best comic book movie ever"" nons...",rw5144576,26 September 2019,gatozangado,It premiered last night in Sydney and I was lu...,10/10,102 out of 206 found this ...,"[spare, ``, best, comic, book, movi, ever, '',...","[it, premier, last, night, sydney, i, lucki, e..."
113,As movie well overrated - acting excellent,rw5175282,9 October 2019,dhllon,The depth of the story was lacking to much foc...,2/10,59 out of 114 found this h...,"[a, movi, well, overr, act, excel]","[the, depth, stori, lack, much, focu, short, a..."


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [125]:
# Write your code here
#(1) Tag parts of Speech (POS) and  calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb)...
nltk.download('averaged_perceptron_tagger')
from collections import Counter
hist = dict()
count_POStag = []
for tokens in cleaned_title:
    title_tagged = nltk.pos_tag(tokens)
    #fd_title = nltk.FreqDist(tag for (token, tag) in title_tagged)
    #print(fd_title)
    count_POStag.append(Counter (tag for tokens, tag in title_tagged))
for tokens in cleaned_content:
    content_tagged = nltk.pos_tag(tokens)
    #fd_title = nltk.FreqDist(tag for (token, tag) in title_tagged)
    #print(fd_title)
    count_POStag. append (Counter (tag for tokens, tag in content_tagged))
    
print(len(count_POStag))
print(count_POStag)
count_VB=0
count_NN=0
count_ADJ=0
count_ADV=0
for pack in count_POStag:
  for item in pack:
    if item =="VB":
      count_VB+=1
    elif item =="NN":
      count_NN+=1
    elif item =="JJ":
      count_ADJ+=1
    elif item =="ADV":
      count_ADV+=1
    else:
      pass
print("The total verbs:", count_VB)
print("The total nouns:", count_NN)
print("The total adjectives:", count_ADJ)
print("The total adverbs:", count_ADV)
#for n in count_POStag:
 # for items in count_POStag
 # hist[key]=hist.get(valu=)+1
  
# (2)Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences.



# for item in title_tagged_counts:
      #print(item)
     # hist[item]=hist.get(item)+1
#print(hist


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
230
[Counter({'NN': 9, 'JJ': 3, 'DT': 1, 'VBD': 1, 'RB': 1, 'VB': 1}), Counter({'NN': 5, 'JJS': 1, 'VB': 1, 'RB': 1, 'VBN': 1}), Counter({'NN': 3, 'JJ': 1}), Counter({'NN': 4, 'RB': 2, 'MD': 1, 'VB': 1, 'JJ': 1, 'POS': 1}), Counter({'DT': 1, 'NN': 1, 'JJ': 1}), Counter({'NN': 2}), Counter({'RB': 1, 'VB': 1, 'WRB': 1, 'JJ': 1, 'VBP': 1}), Counter({'NN': 2, 'VBD': 1, 'JJ': 1}), Counter({'NN': 3, 'DT': 1, 'RB': 1, 'VBD': 1}), Counter({'NN': 6, 'POS': 1, 'JJS': 1, 'VBZ': 1, ':': 1, 'VBD': 1}), Counter({'JJ': 1, 'NN': 1}), Counter({'JJ': 2, 'NN': 1}), Counter({'JJ': 1, 'NN': 1}), Counter({'NN': 2}), Counter({'NN': 4, 'RB': 2, 'VB': 1}), Counter({'JJ': 1, 'NN': 1}), Counter({'NN': 2}), Counter({'VB': 2, 'VBP': 1, 'RB': 1}), Counter({'NN': 2, 'MD': 1, 'VB': 1}), Counter({'NN': 2}), Counter({'NN': 3, 'CD': 1,

**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 